In [ ]:
import hashlib, binascii, os
import random
import time
import paho.mqtt.client as mqtt_client
from auth import ldap_auth

broker="broker.emqx.io"
port =1883
topic = "/python/mqtt"
ldap_server = "ldap.forumsys.com"
username = "cn=read-only-admin,dc=example,dc=com"
password = "password"

# generate client ID with pub prefix randomly
client_id = f'python-mqtt-{random.randint(0, 100)}'

# password hashing
def hash_password(password):
    """Hash a password for storing."""
    salt = hashlib.sha256(os.urandom(60)).hexdigest().encode('ascii')
    pwdhash = hashlib.pbkdf2_hmac('sha512', password.encode('utf-8'), 
                                salt, 100000)
    pwdhash = binascii.hexlify(pwdhash)
    return (salt + pwdhash).decode('ascii')

# print(hash_password(ldap_password))

def connect_mqtt() -> mqtt_client:
    def on_connect(client, userdata, flags, reasonCode, properties=None):
        if reasonCode == 0:
            print("Connected to MQTT Broker!")
        else:
            print("Failed to connect, return code %d\n", reasonCode)

    client = mqtt_client.Client(client_id, protocol=mqtt_client.MQTTv5)
    client.on_connect = on_connect
    client.connect(broker, port, keepalive=60, bind_address="", bind_port=0, clean_start=mqtt_client.MQTT_CLEAN_START_FIRST_ONLY, properties=None)
    return client


def subscribe(client: mqtt_client):
    def on_message(client, userdata, msg):
        print(f"Received `{msg.payload.decode()}` from `{msg.topic}` topic")

    client.subscribe(topic)
    client.on_message = on_message


def run():
    client = connect_mqtt()
    ldap_status = ldap_auth(ldap_server, username, password)
    if(ldap_status):
        print("LDAP server - Authenticated successfully. ");
        subscribe(client)
        client.loop_forever()
    else:
        print("LDAP server - User name and password do not match");
    
    
if __name__ == '__main__':
    run()